In [1]:
import word2vec
import pandas as pd
import numpy as np 
import json
import os
import string
from nltk.tokenize import word_tokenize
import math
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
STOPS = stopwords.words('english')
PROJ_NAME = "BRNN_TOXIC"
MAX_COMMENT_LENGTH = 1500
from collections import Counter
LABELS = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

import word2vec
import pandas as pd
import numpy as np 
import json
import os
from nltk.tokenize import word_tokenize
import math
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
STOPS = stopwords.words('english')
PROJ_NAME = "BRNN_TOXIC"
MAX_COMMENT_LENGTH = 1500
from collections
LABELS = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

In [4]:
with open("config.json",'r') as f:
    config_file = json.load(f)["BASE_CONFIG"]
with open(config_file,'r') as f:
    config = json.load(f)
data_dir=os.path.join(config["data_dir"],PROJ_NAME)
model_dir=os.path.join(config["model_dir"],PROJ_NAME)
out_dir=os.path.join(config["out_dir"],PROJ_NAME)

In [3]:
model = word2vec.load(os.path.join(data_dir,"text.bin"))

In [4]:
len(model.vocab),model.vectors.shape

(41913, (41913, 100))

In [6]:
# find 50 most often negative and 50 most often positive words
df = pd.read_csv(os.path.join(data_dir,"train.csv"))
dfs = {}
null = df.copy()
for name in LABELS:
    dfs[name] = df.loc[(df[name] == 1)]
    null.drop(null[null[name]==1].index,axis=0,inplace=True)
        

total_words = 0
all_words = set()
i = 0
for comment in df["comment_text"]:
    if i%10000 == 0:
        print(i/len(df))
    words = word_tokenize(comment)
    if len(words) < MAX_COMMENT_LENGTH:
        for word in words:
            if word not in STOPS and word not in string.punctuation:
                total_words+=1
                all_words.add(word.lower())
    i+=1
total_words

0.0
0.0626680286518227
0.1253360573036454
0.1880040859554681
0.2506721146072908
0.3133401432591135
0.3760081719109362
0.4386762005627589
0.5013442292145815
0.5640122578664043
0.626680286518227
0.6893483151700497
0.7520163438218724
0.8146843724736951
0.8773524011255178
0.9400204297773405


6789972

In [7]:
frequency_dict = {}
i=0
for c,d in dfs.items():
    print(i)
    word_freq = {}
    for class_comment in d["comment_text"]:
        for word in word_tokenize(class_comment):
            if word in all_words and word not in string.punctuation:
                if word in word_freq.keys():
                    word_freq[word] = word_freq[word]+1
                else:
                    word_freq[word]=1
    frequency_dict[c] = word_freq
    i+=1

0
1
2
3
4
5


In [8]:
null_freq = {}
for class_comment in null["comment_text"]:
        for word in word_tokenize(class_comment):
            if word in all_words and word not in string.punctuation:
                if word in null_freq.keys():
                    null_freq[word] = null_freq[word]+1
                else:
                    null_freq[word]=1

In [9]:
selected = []
for k,v in frequency_dict.items():
    print(k)
    i = 0
    while(i<13):
        c = Counter(v).most_common(1)
        if c[0][0] not in selected and c[0][0] in model.vocab:
            selected.append(c[0][0])
            i+=1   
        v[c[0][0]]=0
i=len(selected)
while(len(selected)<100):
    c = Counter(null_freq).most_common(1)
    if c[0][0] not in selected and c[0][0] in model.vocab:
        selected.append(c[0][0])
        i+=1
    null_freq[c[0][0]]=0
selected

toxic
severe_toxic
obscene
threat
insult
identity_hate


['you',
 'a',
 'the',
 'to',
 'and',
 "''",
 'is',
 'of',
 'your',
 'that',
 '``',
 'it',
 'are',
 'fuck',
 'ass',
 'faggot',
 'yourself',
 "'re",
 'u',
 'fucksex',
 'yourselfgo',
 'i',
 'shit',
 "n't",
 'my',
 'fucking',
 'in',
 'do',
 'on',
 'for',
 'me',
 "'s",
 'have',
 '...',
 'this',
 'not',
 'with',
 'be',
 'like',
 'going',
 'am',
 'kill',
 'will',
 'murder',
 'die',
 "'m",
 'if',
 'get',
 'so',
 'all',
 'hope',
 'go',
 'moron',
 'an',
 'hi',
 'know',
 'as',
 'off',
 'what',
 'who',
 'stupid',
 'bitch',
 'just',
 'up',
 'about',
 'gay',
 'nigger',
 'licker',
 'ca',
 'down',
 'or',
 'keep',
 'people',
 'nigga',
 'they',
 'but',
 'by',
 'from',
 'article',
 'page',
 'at',
 'can',
 'would',
 'there',
 'talk',
 'one',
 'which',
 'been',
 'should',
 'any',
 'more',
 'no',
 'he',
 'other',
 'some',
 'we',
 'think',
 'here',
 'see',
 'his']

In [10]:
def adjust_class_balance(df: pd.DataFrame, interested_labels, thresh):
    dfs = {}
    null = df.copy()
    for name in interested_labels:
        dfs[name] = df.loc[(df[name] == 1)]
        null.drop(null[null[name]==1].index,axis=0,inplace=True)
        
    print("NULL:", 100*(len(null)/len(df)))
    for name, d in dfs.items():
        print("Initial percentage of DF for", name, "is", 100*(len(d)/len(df)))
    
    print("Each label will now have at least", thresh*100,"% of the origional df size")
    adjusted_df = null.sample(int(thresh*len(df))) # get a subsample of null cases
    

    for n, d in dfs.items():
        i=0
        for times in range(math.ceil((thresh/(len(d)/len(df))+1))):
            adjusted_df = adjusted_df.append(d)
            i+=1
        print(n,"upsampled",i,"times")
    return adjusted_df
a_df = adjust_class_balance(df, LABELS, 1/(len(LABELS)+1))

NULL: 89.83211235124176
Initial percentage of DF for toxic is 9.584448302009765
Initial percentage of DF for severe_toxic is 0.9995550569965721
Initial percentage of DF for obscene is 5.2948217407925
Initial percentage of DF for threat is 0.2995531769557125
Initial percentage of DF for insult is 4.936360616904074
Initial percentage of DF for identity_hate is 0.8804858025581089
Each label will now have at least 14.285714285714285 % of the origional df size
toxic upsampled 3 times
severe_toxic upsampled 16 times
obscene upsampled 4 times
threat upsampled 49 times
insult upsampled 4 times
identity_hate upsampled 18 times


In [11]:
model["hello"]

array([-0.14757682, -0.04240734,  0.19148824,  0.00098323,  0.0573453 ,
       -0.05768183, -0.15212771, -0.07680046,  0.03413858,  0.17289601,
       -0.17418103, -0.00899368,  0.16075474, -0.02236715,  0.0478254 ,
       -0.09537709,  0.12551017,  0.13849607,  0.08788317, -0.21410128,
       -0.00878321, -0.04462972, -0.01825777, -0.13360611,  0.06181644,
       -0.01524204,  0.0692144 , -0.06240428,  0.09520414, -0.0216194 ,
       -0.1535608 ,  0.02832055, -0.22033127,  0.08884833,  0.12681127,
       -0.13558899,  0.04884369,  0.06884765, -0.03505035,  0.00168696,
        0.08216715,  0.05765036, -0.1474108 , -0.08059205,  0.05477362,
       -0.12545238,  0.10483534,  0.05306536, -0.05766199,  0.0028046 ,
       -0.00083108,  0.14653622, -0.15248394, -0.09605292,  0.1178815 ,
       -0.14056268,  0.04231599, -0.14468089, -0.05952892, -0.03034033,
       -0.05168787, -0.00610198,  0.10717586,  0.07031706, -0.03640174,
       -0.08431084,  0.01758727, -0.00465056, -0.01382627, -0.02

In [ ]:
def data_from_comment(comment):
    vec = []
    clean = [ x.lower() for x in word_tokenize(comment)]
    for sel in selected:
        if sel in clean:
            vec.append(model[sel])
        else:
            vec.append(np.zeros(100))
    return vec
    
        
        
a_data = [] 
a_labels = []
for index, row in df.iterrows():
    if index %1000 ==0:
        print(index/len(df)*100)
    included = False
    temp_data = data_from_comment(row["comment_text"])
    for counter, include in enumerate(row[LABELS].values):
        if include == 1:
            one_hot = np.zeros(len(LABELS))
            one_hot[counter] = 1
            a_labels.append(one_hot)
            a_data.append(temp_data)
            included = True
            
    if not included:
        a_labels.append(np.zeros(len(LABELS)))
        a_data.append(temp_data)
    

0.0
0.6266802865182269
1.2533605730364539
1.8800408595546811
2.5067211460729077
3.1334014325911346
3.7600817191093623
4.386762005627589
5.0134422921458155
5.640122578664044
6.266802865182269
6.893483151700497
7.520163438218725
8.14684372473695
8.773524011255178
9.400204297773405
10.026884584291631
10.65356487080986
11.280245157328087
11.906925443846312
12.533605730364538
13.160286016882766
13.786966303400995
14.413646589919221
15.04032687643745
15.667007162955676
16.2936874494739
16.92036773599213
17.547048022510356
18.173728309028583
18.80040859554681
19.427088882065036
20.053769168583262
20.680449455101492
21.30712974161972
21.933810028137945
22.560490314656175
23.187170601174397
23.813850887692624
24.440531174210854
25.067211460729077
25.693891747247307
26.320572033765533
26.94725232028376
27.57393260680199
28.200612893320216
28.827293179838442
29.45397346635667
30.0806537528749
30.707334039393125
31.33401432591135
31.960694612429574
32.5873748989478
33.21405518546603
33.84073547198

In [ ]:
X = np.array(a_data)
Y = np.array(a_labels)

In [3]:
print(X.shape,Y.shape)
np.save(os.path.join(data_dir,"X_ALL.npy"),X)
np.save(os.path.join(data_dir,"Y_ALL.npy"),Y)

Signature: np.save(file, arr, allow_pickle=True, fix_imports=True)
Docstring:
Save an array to a binary file in NumPy ``.npy`` format.

Parameters
----------
file : file, str, or pathlib.Path
    File or filename to which the data is saved.  If file is a file-object,
    then the filename is unchanged.  If file is a string or Path, a ``.npy``
    extension will be appended to the file name if it does not already
    have one.
arr : array_like
    Array data to be saved.
allow_pickle : bool, optional
    Allow saving object arrays using Python pickles. Reasons for disallowing
    pickles include security (loading pickled data can execute arbitrary
    code) and portability (pickled objects may not be loadable on different
    Python installations, for example if the stored objects require libraries
    that are not available, and not all pickled data is compatible between
    Python 2 and Python 3).
    Default: True
fix_imports : bool, optional
    Only useful in forcing objects in obj